In [2]:
from datetime import datetime
from os.path import basename, exists
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import requests
import os
import json
from datetime import datetime
np.seterr(divide='ignore', invalid='ignore')
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [15]:
day_mean = 10
today = datetime.now()
yesterday = datetime.now() - timedelta(1)
URL = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{0:%Y}-{0:%m}-{0:%d}.xlsx".format(today)
FILE = "Data/"+basename(URL)

print('Database ', end='')
if not exists(FILE):
    with open(FILE, 'wb') as f:
        f.write(requests.get(URL, allow_redirects=False).content)
        print(' Downloaded')
else:
    print(' Exists')

WW_DATA = pd.read_excel(FILE)
WW_DATA['dateRep'] = pd.to_datetime(WW_DATA['dateRep'], format='%d/%m/%Y')
WW_DATA = WW_DATA.drop(WW_DATA[WW_DATA['countriesAndTerritories']=='China'].index)

# DATA CORRECTION ON SPAIN 25 OF MAY
# WW_DATA.loc[(WW_DATA.dateRep == datetime(year=2020, month=5, day=25)) & (WW_DATA['countriesAndTerritories']=='Spain'), 'deaths'] = 100
# WW_DATA.loc[(WW_DATA.dateRep == datetime(year=2020, month=5, day=25)) & (WW_DATA['countriesAndTerritories']=='Spain'), 'cases'] = 372

Database  Exists


In [14]:
def smoother(array, window=5, pad=True):
    """
    Regresa un vector de la misma longitud que el introducido, dependiendo
    del valor de la ventana ajusta los extremos al promedio de los valores
    que puede ocupar.
    
    ó, si pad = False
    Regresa sólo el vector promediado con los (window-1) datos en los ext-
    tremos acortados.
    
    VALOR MÍNIMO DE VENTANA ES 3
    """
    serie = pd.Series(array)
    data = list(serie.rolling(window=window).mean())
    numb = sum(np.isnan(data)*1)
    if not pad:
        return data[numb:]
    lower, upper = int(np.ceil(numb/2)), int(np.floor(numb/2))
    ini = [np.mean(array[:i+upper]) for i in range(lower)]
    end = [np.mean(array[(len(array)-(lower+i)):]) for i in range(upper)]
    return np.array(ini+data[numb:]+end)

def data2log(data, day_mean, ret_smth=False, pad=True):
    """
    Aplica logaritmo a una serie de datos, transforma valores inválidos a 0
    los datos de entrada deben ser positivos
    """
    ntc = smoother(data, day_mean, pad=pad)
    dt = np.divide(ntc[1:], ntc[:-1])
    dt = np.where(np.logical_or(np.isnan(dt), np.isinf(dt)), 0.0, dt)
    if ret_smth:
        return ntc[:-1], np.where(dt != 0, np.log(dt), 0)
    else:
        return np.where(dt != 0, np.log(dt), 0)

def trim_first_zero(vector1, vector2=None, buffer=0):
    """
    Quita la cadena de 0's al inicio de los datos.
    Si se ingresa un segundo vector, regresa ambos vectores del mismo
    tamaño, permitiendo ajustar los datos de los dos ectores.
    """
    for idx, val in enumerate(vector1):
        if val > 0:
            if vector2 is not None:
                return vector1[(idx-buffer):], vector2[(idx-buffer):]
            else:
                return vector1[(idx-buffer):]

In [4]:
def country_selector(DF, COUNTRIES):
    container = dict()
    print(f'\nSelecting bundle of countries: \n')
    for country in COUNTRIES:
        country_data = WW_DATA[WW_DATA.countriesAndTerritories == country].sort_values(by='dateRep',
                                                                            ascending=True,
                                                                             ignore_index=True)
    new_data = country_data[:-1][["cases", "deaths"]]

    new_data["vel_cases"] = data2log(country_data.cases, 10)
    new_data["smooth_cases"] = smoother(new_data.cases, 10)

    new_data["vel_deaths"] = data2log(country_data.deaths, 10)
    new_data["smooth_deaths"] = smoother(new_data.deaths, 10)

    try:
        container[country] = new_data.to_dict(orient="registers")
    except (NameError, AttributeError) as r:
        print(country, len(new_data), r)
    with open("./covid_data_top.js", "w+") as f:
        f.write("export var covid_data_top = ")
        json.dump(container, f)
        f.write(";")
    print("Top Countries js file Ready!")
    return new_data      

In [46]:
COUNTRIES = [
    'United_States_of_America',
    'Belgium',
    'Spain',
    'Italy',
    'Germany',
]
PAISES = [
    'Estados Unidos',
    'Belgica',
    'España',
    'Italia',
    'Alemania'
]
new_data = country_selector(WW_DATA, COUNTRIES)


Selecting bundle of countries: 

Top Countries js file Ready!


In [11]:
country_data = WW_DATA[WW_DATA.countriesAndTerritories == 'United_States_of_America'].sort_values(
        by='dateRep',
        ascending=True,
        ignore_index=True)

country_data.head()


,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp
0,2019-12-31,31,12,2019,0,0,United_States_of_America,US,USA,329064917.0,America
1,2020-01-01,1,1,2020,0,0,United_States_of_America,US,USA,329064917.0,America
2,2020-01-02,2,1,2020,0,0,United_States_of_America,US,USA,329064917.0,America
3,2020-01-03,3,1,2020,0,0,United_States_of_America,US,USA,329064917.0,America
4,2020-01-04,4,1,2020,0,0,United_States_of_America,US,USA,329064917.0,America


In [60]:
letalidad = (np.sum(country_data.deaths)/np.sum(country_data.cases))*100
population = country_data.popData2019[0]

new_data = pd.DataFrame()

#  Nuevos Casos
ntc, rtc = data2log(country_data.cases, day_mean, ret_smth=True)
xc, yc = trim_first_zero(ntc, rtc)

xc = smoother((xc/population)*10**5)
yc = smoother(yc)

new_data["Tasa_Incid"] = xc
new_data["r(t)_casos"] = yc

# Defunciones
ntd, rtd = data2log(country_data.deaths, day_mean, ret_smth=True)
dif=abs(len(ntd)-len(xc))

xd = ntd[dif: len(ntd)]
yd = rtd[dif:len(rtd)]

xd=smoother((xd/population)*10**6)
yd=smoother(yd)

new_data["Tasa_defun"] = xd
new_data["r(t)_deaths"] = yd

new_data


,Tasa_Incid,r(t)_casos,Tasa_defun,r(t)_deaths
0,0.000030,0.000000,0.000000,0.000000
1,0.000030,0.000000,0.000000,0.000000
2,0.000036,0.138629,0.000000,0.000000
3,0.000043,0.321888,0.000000,0.000000
4,0.000067,0.321888,0.000000,0.000000
...,...,...,...,...
166,14.026339,0.030013,2.117880,-0.054631
167,14.447095,0.016266,1.996749,-0.065354
168,14.677442,0.006320,1.857723,-0.066853
169,14.921122,-0.024667,1.863397,-0.070551


In [10]:
# All Countries 
container = dict()
for country in WW_DATA["countriesAndTerritories"].unique():
    
    country_data = WW_DATA[WW_DATA.countriesAndTerritories == country].sort_values(
        by='dateRep',
        ascending=True,
        ignore_index=True)

    letalidad = (np.sum(country_data.deaths)/np.sum(country_data.cases))*100
    population = country_data.popData2019[0]

    new_data = pd.DataFrame()

    #  Nuevos Casos
    ntc, rtc = data2log(country_data.cases, day_mean, ret_smth=True)
    xc, yc = trim_first_zero(ntc, rtc)

    xc = smoother((xc/population)*10**5)
    yc = smoother(yc)

    new_data["Tasa_Incid"] = xc
    new_data["r(t)_casos"] = yc

    # Defunciones
    ntd, rtd = data2log(country_data.deaths, day_mean, ret_smth=True)
    dif=abs(len(ntd)-len(xc))

    xd = ntd[dif: len(ntd)]
    yd = rtd[dif:len(rtd)]

    xd=smoother((xd/population)*10**6)
    yd=smoother(yd)

    new_data["Tasa_defun"] = xd
    new_data["r(t)_deaths"] = yd

    try:
        container[country] = new_data.to_dict(orient="registers")
    except (NameError, AttributeError) as r:
        print(country, len(new_data), r)
with open("./covid_data_ww.js", "w+") as f:
    f.write("export var covid_data_ww = ")
    json.dump(container, f)
    f.write(";")

In [23]:
new_data


,dateRep,cases,deaths,vel_cases,smooth_cases,vel_deaths,smooth_deaths
0,2019-12-31T00:00:00,0,0,0.000000,0.000000,0.000000,0.000000
1,2020-01-01T00:00:00,0,0,0.000000,0.000000,0.000000,0.000000
2,2020-01-02T00:00:00,0,0,0.000000,0.000000,0.000000,0.000000
3,2020-01-03T00:00:00,0,0,0.000000,0.000000,0.000000,0.000000
4,2020-01-04T00:00:00,0,0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
173,2020-06-26T00:00:00,197,9,0.096321,206.700000,-0.064539,8.000000
174,2020-06-27T00:00:00,240,7,0.250533,264.600000,-0.097980,7.200000
175,2020-06-28T00:00:00,283,7,-0.055912,249.166667,0.052518,7.333333
176,2020-06-29T00:00:00,305,5,-0.055503,235.857143,0.016475,7.571429


In [28]:
fechas = list(range(15, 25))
a = list(range(0, 10))
b = list(range(10, 20))
c = list(range(20, 30))
l = [a, b, c]
nombres = ["Uno", "Dos", "Tres"]
dic = {"fechas": fechas}
dic.update({nombres[s]: l[s] for s in range(3)})
dic

{'fechas': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 'Uno': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'Dos': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 'Tres': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]}

In [29]:
pd.DataFrame(dic)

,fechas,Uno,Dos,Tres
0,15,0,10,20
1,16,1,11,21
2,17,2,12,22
3,18,3,13,23
4,19,4,14,24
5,20,5,15,25
6,21,6,16,26
7,22,7,17,27
8,23,8,18,28
9,24,9,19,29
